In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.circuit.classical import expr

from qiskit.transpiler import PassManager
from LogicalQ.Transpilation.DecomposeIfElseOps import DecomposeIfElseOps

from pytket.extensions.qiskit import qiskit_to_tk

In [ ]:
def cbit_and(cbits, values):
    result = expr.bit_not(cbits[0]) if values[0] == 0 else expr.lift(cbits[0])
    for n in range(len(cbits)-1):
        result = expr.bit_and(result, expr.bit_not(cbits[n+1])) if values[n+1] == 0 else expr.bit_and(result, cbits[n+1])
    return result

def cbit_xor(cbits):
    result = expr.lift(cbits[0])
    for n in range(len(cbits)-1):
        result = expr.bit_xor(result, cbits[n+1])
    return result

In [ ]:
qreg = QuantumRegister(1)
qreg_setter = QuantumRegister(2)
creg_A = ClassicalRegister(1)
creg_B = ClassicalRegister(1)
creg_C = ClassicalRegister(1)

qc = QuantumCircuit(qreg, qreg_setter, creg_A, creg_B, creg_C)
qc.x(qreg_setter[1])

with qc.if_test(cbit_and([creg_A[0], creg_B[0]], [1, 0])) as _else:
    qc.measure(qreg_setter[1], creg_C[0])
with _else:
    qc.measure(qreg_setter[0], creg_C[0])

with qc.if_test(cbit_xor([creg_A[0], creg_B[0]])) as _else:
    qc.measure(qreg_setter[1], creg_C[0])
with _else:
    qc.measure(qreg_setter[0], creg_C[0])

qc.draw("mpl")

In [ ]:
pm = PassManager([DecomposeIfElseOps()])
qc_transpiled = qc
qc_transpiled = pm.run(qc_transpiled)
qc_transpiled.draw("mpl")

In [ ]:
circuit = qiskit_to_tk(qc_transpiled)